In [ ]:
# Set this variable yourself.
running_on_colab = False
# Store data as reduced density matrix `rho` or eigenvector tuple `EVW`.
rho_or_EVW = 'EVW'

# Machine Learning of Many Body Localization

## Introduction

Use exact diagonalization to obtain a few eigenstates near energy $E = 0$ from the Heisenberg model with a
random field, 

\begin{equation}
    H = J \sum_i \vec{S}_{i} \cdot \vec{S}_{i+1} - \sum_i h_i S^z_i
\end{equation}

, where the values of the field $ h_i \in [-W, W] $ are chosen from a uniform random distribution with a "disorder strength" $W$ (with moderate system sizes $L \approx 12$). 

The exciting property of this model is that it is believed to undergo a phase transition from an extended phase (small $W$) to a localized phase (large $W$). 

We will use ML to detect this transition: Pick a number of eigenstates that are near energy $E = 0$ and obtain the reduced density matrices $\rho^A$, where $A$ is a region of $n$ consecutive spins (a few hundred to thousands eigenstates for different disorder realizations). 

Now use the density matrices for $W = 0.5 J$ and $W = 8.0 J$ to train a neural network (just interpret the entries of $\rho^A$ as an image with $2^n \times 2^n$ pixel). 
Then use this network and study the output of the neural network for different $W$. 

How does the results depend on system size $L$ and block size $n$? 
At which $W_c$ do you expect the transition to occur?

### Wikipedia:  
[Many body localization](https://en.wikipedia.org/wiki/Many_body_localization)  
[Localization protected quantum order](https://en.wikipedia.org/wiki/Localization_protected_quantum_order)  

Many-body localization (MBL) is a dynamical phenomenon which leads to the breakdown of equilibrium statistical mechanics in isolated many-body systems. Such systems never reach local thermal equilibrium, and retain local memory of their initial conditions for infinite times.

MBL was first proposed by P.W. Anderson in 1958 as a possibility that could arise in strongly disordered quantum systems. The basic idea was that if particles all live in a random energy landscape, then any rearrangement of particles would change the energy of the system. Since energy is a conserved quantity in quantum mechanics, such a process can only be virtual and cannot lead to any transport of particle number or energy.  

The process of thermalization erases local memory of the initial conditions. In textbooks, thermalization is ensured by coupling the system to an external environment or "reservoir," with which the system can exchange energy. What happens if the system is isolated from the environment, and evolves according to its own Schrödinger equation? Does the system still thermalize?

Quantum mechanical time evolution is unitary and formally preserves all information about the initial condition in the quantum state at all times.

This question can be formalized by considering the quantum mechanical density matrix ρ of the system. If the system is divided into a subregion A (the region being probed) and its complement B (everything else), then all information that can be extracted by measurements made on A alone is encoded in the reduced density matrix $\rho_A = Tr_B (\rho(t))$. If in the long time limit $\rho_A(t)$ approaches a thermal density matrix at a temperature set by the energy density in the state, then the system has "thermalized," and no local information about the initial condition can be extracted from local measurements. This process of "quantum thermalization" may be understood in terms of B acting as a reservoir for A. In this perspective, the entanglement entropy $ S = - Tr \rho_A log \rho_A $ of a thermalizing system in a pure state plays the role of thermal entropy. Thermalizing systems therefore generically have extensive or "volume law" entanglement entropy at any non-zero temperature.

In contrast, if $\rho_A(t)$ fails to approach a thermal density matrix even in the long time limit, and remains instead close to its initial condition $\rho_A(0)$, then the system retains forever a memory of its initial condition in local observables. This latter possibility is referred to as "many body localization," and involves B failing to act as a reservoir for A. Eigenstates of systems exhibiting MBL do not obey the ETH, and generically follow an "area law" for entanglement entropy (i.e. the entanglement entropy scales with the surface area of subregion A).

In thermalizing systems, energy eigenstates have volume law entanglement entropy. In MBL systems, energy eigenstates have area law entanglement entropy.

In thermalizing systems, entanglement entropy grows as a power law in time starting from low entanglement initial conditions. In MBL systems, entanglement entropy grows logarithmically in time starting from low entanglement initial conditions.

In thermalizing systems, the dynamics of out-of-time-ordered correlators forms a linear light cone which reflects the ballistic propagation of information. In MBL systems, the light cone is logarithmic.

What's more, while individual eigenstates aren't themselves experimentally accessible, order in eigenstates nevertheless has measurable dynamical signatures. The eigenspectrum properties change in a singular fashion as the system transitions between from one type of MBL phase to another, or from an MBL phase to a thermal one---again with measurable dynamical signatures.

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

os.environ['running_on_colab'] = str(running_on_colab)
# running_on_colab = (os.getenv('running_on_colab', 'False') == 'True')

if running_on_colab:
    data_root             = 'drive/MyDrive/Colab Data/MBL/'
    sys.path.append(data_root)
else:
    data_root             = './'

# Store data as reduced density matrix `rho` or eigenvector tuple `EVW`.
os.environ['rho_or_EVW'] = str(rho_or_EVW)
# running_on_colab = (os.getenv('rho_or_EVW', 'EVW') == 'rho')

from file_io import *
from data_gen import *

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.ticker import MaxNLocator

dpi = 100
fig_w = 1280
fig_h = 640

%matplotlib inline

In [ ]:
if running_on_colab:
    !cat /proc/cpuinfo

In [ ]:
if running_on_colab:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
if running_on_colab:
    !pip install pytorch_lightning==0.7.6 torchsummary==1.5.1

## Test execution time of Exact Diagonalization

### Test solving a single Hamiltonian

In [ ]:
L = 10
W = 0.5
J = 1
periodic = False
num_Hs = 10

H = build_H(L, W, J, periodic)
E, V = ED(H.toarray())
print('2^L: {}'.format(2**L))
print('#eigenvalues: {}'.format(len(E)))
print('E.shape: {}'.format(E.shape))
print('V.shape: {}'.format(V.shape))

In [ ]:
%%timeit
# Solve using numpy's dense solver
E, V = ED(H.toarray())
E0, V0 = select_N_eigenvalues(E, V, 20)

In [ ]:
%%timeit
# Solve using scipy's sparse solver instead.
E1, V1 = ED_sparse(H, 20)

In [ ]:
# Solve using numpy's dense solver
E, V = ED(H.toarray())
E0, V0 = select_N_eigenvalues(E, V, 20)

# Solve using scipy's sparse solver instead.
E1, V1 = ED_sparse(H, 20)

In [ ]:
# Check eigenvalues are sorted correctly, and eigenvectors are selected along the correct axis.
V0_norm = []
V1_norm = []
for i in range(len(E0)):
    V0_norm.append(np.linalg.norm(V0[:,i]))
    V1_norm.append(np.linalg.norm(V1[:,i]))

for E0i, E1i, V0i, V1i in zip(E0, E1, V0_norm, V1_norm):
    print('Numpy: {:+.12f} | Numpy norm: {:.16f}'.format(E0i, V0i))
    print('Scipy: {:+.12f} | Scipy norm: {:.16f}'.format(E1i, V1i))

assert np.allclose(V0i, V1i), 'Eigenvectors evaluated using Numpy and Scipy should be identical.'

### Test solving multiple Hamiltonians

In [ ]:
Hs = build_Hs(L, W, J, periodic, num_Hs)
Es, Vs = EDs(Hs)
E0s, V0s = EDs_sparse(Hs, 20)

### Test execution time, numpy vs scipy

In [ ]:
# Test execution time: numpy.
J  = 1                      # Always = 1
Ws = [8]                    # Disorder strength W.
Ls = list(range(8,12))      # System size L.
ns = [1]*len(Ls)            # Number of samples for each L.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 20                 # Number of eigenvalues near zero to save.

for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            batch_generate_ED_data(L, W, J, p, num_Hs, num_EV, save_data=False, npsp='np')
            exec_time = time.time() - start_time
            et.append(exec_time)
            print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(fig_w/dpi,fig_h/dpi), dpi=dpi, squeeze=False)

base = 10
axes[0,0].plot(Ls, et)
axes[0,1].plot(np.array(Ls), np.log(et) / np.log(base))

axes[0,0].set_title('Numpy Execution time vs System size')
axes[0,1].set_title('Log(Numpy Execution time) vs System size')
axes[0,0].set_ylabel('Execution time')
axes[0,1].set_ylabel('Log(Execution time)')

for axe in axes:
    for ax in axe:
        ax.set_xlabel('System size L')
        # ax.legend(loc='best')
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
# Test execution time: scipy.
J  = 1                      # Always = 1
Ws = [8]                    # Disorder strength W.
Ls = list(range(8,16))      # System size L.
ns = [1]*len(Ls)            # Number of samples for each L.
ps = [False]                # Periodic or not.
et = []                     # Execution time.
num_EV = 20                 # Number of eigenvalues near zero to save.

for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            batch_generate_ED_data(L, W, J, p, num_Hs, num_EV, save_data=False, npsp='sp')
            exec_time = time.time() - start_time
            et.append(exec_time)
            print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(fig_w/dpi,fig_h/dpi), dpi=dpi, squeeze=False)

base = 10
axes[0,0].plot(Ls, et)
axes[0,1].plot(np.array(Ls), np.log(et) / np.log(base))

axes[0,0].set_title('SciPy Execution time vs System size')
axes[0,1].set_title('Log(SciPy Execution time) vs System size')
axes[0,0].set_ylabel('Execution time')
axes[0,1].set_ylabel('Log(Execution time)')

for axe in axes:
    for ax in axe:
        ax.set_xlabel('System size L')
        # ax.legend(loc='best')
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
# Sample parameters.
J  = 1                                 # Always = 1
Ws = [0.5, 8] * 10 + [i/2 for i in range(1*2, 10*2)] # Disorder strength W.
Ls = [   8,   9,  10,  11, 12, 13, 14] # System size L.
ns = [1000, 500, 250, 100, 50, 20, 10] # Number of samples for each L.
ps = [False, True]                     # Periodic or not.

In [ ]:
# Test file size. It's not feasible to store ALL eigenvectors.
J  = 1             # Always = 1
Ws = [0.5, 8] * 10 + [i/2 for i in range(1*2, 10*2)] # Disorder strength W.
Ls = [   8]        # System size L.
ns = [1000]        # Number of samples for each L.
ps = [False, True] # Periodic or not.

fs = 0
for L, num_Hs in zip(Ls, ns):
    for W in Ws:
        for p in ps:
            start_time = time.time()
            # batch_generate_ED_data(L, W, J, p, num_Hs)
            fs += 1
            exec_time = time.time() - start_time
            # print('Computed: L={:02d} | W={:.2f} | periodic={: <5}. Execution took {: 8.2f}s or {: 6.2f}min'.format(L, W, str(p), exec_time, exec_time/60))
print('Eigenvectors `Vs` dominates the file size. Assume 1000 samples are generated for each W, for around 20 Ws:')
print('For L=8, each Es file is 2MB, Hs is 23MB, Vs is 1000MB.')
print('Estimate file size of each run is thus {}MB'.format(fs*(2+23+1000)))

In [ ]:
print('The full eigenvectors `Vs` dominates the file size.')
for i in range(5):
    print('For L = {:2d}, Vs is {:3d} MB.'.format(8+i, 4**i))

## Visualize eigenvectors for different W

In [ ]:
num = 4
num_plots = num * num # Number of rho_A to display.
base_samples = num_plots // 2 * 128

In [ ]:
# Test drawing images of eigenvectors.
J  = 1                      # Always = 1
L  = 10                     # System size L.
Ls = [L]                    # System sizes Ls.
ps = [False]                # Periodic or not.
num_EV = 1                  # Number of eigenvalues near zero to save.

In [ ]:
# Training data.
Ws_main = [0.5, 8]          # Disorder strength W.
Hs = [base_samples]         # Number of samples per L per W.
et = []                     # Execution time.
EVWs_main = []

for L, num_Hs in zip(Ls, Hs):
    for p in ps:

        start_time = time.time()

        EVWsi = batch_gen_EVW_data_main(L, Ws_main, J, p, num_Hs, num_EV, save_data=False)
        EVWs_main.extend(EVWsi)

        exec_time = time.time() - start_time
        et.append(exec_time)
        print('Computed: L={:02d} | periodic={: <5}.'.format(L, str(p)))
        print('Execution took {: 8.2f}s or {: 6.2f}min.'.format(exec_time, exec_time/60))


In [ ]:
# # Random data.
# Ws_rand = np.random.uniform(0.1, high=9.9, size=(2 * base_samples,)) # Disorder strength W.
# Hs = [1]                    # Number of samples per L per W.
# et = []                     # Execution time.
# EVWs_rand = []

# for L, num_Hs in zip(Ls, Hs):
#     for p in ps:

#         start_time = time.time()

#         EVWsi = batch_gen_EVW_data_rand(L, Ws_rand, J, p, num_Hs, num_EV, save_data=False)
#         EVWs_rand.extend(EVWsi)

#         exec_time = time.time() - start_time
#         et.append(exec_time)
#         print('Computed: L={:02d} | periodic={: <5}.'.format(L, str(p)))
#         print('Execution took {: 8.2f}s or {: 6.2f}min.'.format(exec_time, exec_time/60))


In [ ]:
print(len(EVWs_main))       # = base_samples * len(Ws) * num_EV
print(len(EVWs_main[0]))    # = 3
print(len(EVWs_main[0][1])) # = 2**L

# print(len(EVWs_rand))       # = base_samples * len(Ws) * num_EV
# print(len(EVWs_rand[0]))    # = 3
# print(len(EVWs_rand[0][1])) # = 2**L

In [ ]:
Wext_Vs_main = []
Wloc_Vs_main = []

for E, V, W in EVWs_main:

    if W == 0.5:
        Wext_Vs_main.append(V)
    elif W == 8:
        Wloc_Vs_main.append(V)

Wext_Vs_main = np.array(Wext_Vs_main)
Wloc_Vs_main = np.array(Wloc_Vs_main)
print(Wext_Vs_main.shape)
print(Wloc_Vs_main.shape)

# Reshape due to `num_EV`.
Wext_Vs_main = Wext_Vs_main.reshape(-1, 2**L)
Wloc_Vs_main = Wloc_Vs_main.reshape(-1, 2**L)
print(Wext_Vs_main.shape)
print(Wloc_Vs_main.shape)

In [ ]:
# Wext_Vs_rand = []
# Wloc_Vs_rand = []

# for E, V, W in EVWs_rand:

#     if W <= 3.0:
#         Wext_Vs_rand.append(V)
#     else:
#         Wloc_Vs_rand.append(V)

# Wext_Vs_rand = np.array(Wext_Vs_rand)
# Wloc_Vs_rand = np.array(Wloc_Vs_rand)
# print(Wext_Vs_rand.shape)
# print(Wloc_Vs_rand.shape)

# # Reshape due to `num_EV`.
# Wext_Vs_rand = Wext_Vs_rand.reshape(-1, 2**L)
# Wloc_Vs_rand = Wloc_Vs_rand.reshape(-1, 2**L)
# print(Wext_Vs_rand.shape)
# print(Wloc_Vs_rand.shape)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(fig_w/dpi,fig_h/dpi), dpi=dpi, squeeze=False)
fig.suptitle('Visualize training eigenvectors ($W=0.5,W=8.0$)', fontsize=16)

axes[0,0].imshow(np.abs(Wext_Vs_main), vmin=0, vmax=1e-1)
axes[0,0].set_title('Extended phase ($W=0.5$)')
axes[0,0].annotate('W={:3.1f}'.format(0.5), (0.5,0.5), xycoords='axes fraction', ha='center', color='w', fontsize=14)

axes[0,1].imshow(np.abs(Wloc_Vs_main), vmin=0, vmax=1e-1)
axes[0,1].set_title('Localized phase ($W=8.0$)')
axes[0,1].annotate('W={:3.1f}'.format(8.0), (0.5,0.5), xycoords='axes fraction', ha='center', color='w', fontsize=14)

for axe in axes:
    for ax in axe:
        # ax.legend(loc='best')
        ax.xaxis.set_ticklabels([])
        ax.yaxis.set_ticklabels([])
        # ax.xaxis.set_visible(False)
        # ax.yaxis.set_visible(False)
        ax.set_xlabel('Eigenvector coefficients')
        ax.set_ylabel('Eigenvector samples')

fig.tight_layout()
fig.subplots_adjust(top=0.9)

## Batch generate data (execution part)

In [ ]:
print('Estimated runtime to generate 10,000 samples per W:')
for i in range(7):
    print('For L = {:2d}, est runtime {:3d} min or {: 2.2f} hrs.'.format(8+i, 7 * 2**i, 7 * 2**i / 60))
print(' ')

print('Estimated runtime to generate 100,000 samples per W:')
for i in range(7):
    print('For L = {:2d}, est runtime {:4d} min or {:5.2f} hrs.'.format(8+i, 70 * 2**i, 70 * 2**i / 60))
print(' ')

print('It will therefore be safer to break down generation to segments of 1-hr:')
for i in range(7):
    print('For L = {:2d}, separate execution into {:2d} batches with {:6d} samples each.'.format(8+i, 2**i, 100000 // 2**i))

In [ ]:
# Batch generate eigenvectors.

k = 5
batches = 10
batch_resume = 2
base_sample = 10000 // k // batches # Divide by k (num_EV)
rand_sample = 100           # Samples per random W.
Ws_main = [0.5, 8]          # Disorder strength W.
Ws_rand = np.random.uniform(0.1, high=7.9, size=(2 * base_sample // rand_sample,))
J  = 1                      # Always = 1
Ls = list(range(8,13,1))    # System sizes L.
ps = [False, True]          # Periodic or not.
et = []                     # Execution time.
Hs_main = [base_sample]*len(Ls) # Number of samples per L per W.
Hs_rand = [rand_sample]*len(Ls) # Number of samples per L per W.
num_EVs = [k]               # Number of eigenvalues near zero to save.

for i in range(batches):

    if i < batch_resume:
        tqdm.write('{} | Processing batch {:03d} of {:d}:'.format(dt(), i+1, batches)) #, flush=True)
        tqdm.write('{} | Batch {:03d} skipped.'.format(dt(), i+1, batches)) #, flush=True)
        tqdm.write(' ') #, flush=True)
        continue

    tqdm.write('='*60) #, flush=True)
    tqdm.write('{} | Processing batch {:03d} of {:d}:'.format(dt(), i+1, batches)) #, flush=True)
    tqdm.write(' ') #, flush=True)
    for L, num_Hs_m, num_Hs_r in zip(Ls, Hs_main, Hs_rand):

        tqdm.write('='*40) #, flush=True)
        for num_EV in num_EVs:
            for p in ps:
                start_time = time.time()

                tqdm.write('{} | Generating training data for L={:02d} | num_EV={} | periodic={: <5}...'.format(dt(), L, num_EV, str(p))) #, flush=True)
                batch_gen_EVW_data_main(L, Ws_main, J, p, num_Hs_m, num_EV, save_data=True)
                tqdm.write('{} | Generating random data for L={:02d} | num_EV={} | periodic={: <5}...'.format(dt(), L, num_EV, str(p))) #, flush=True)
                batch_gen_EVW_data_rand(L, Ws_rand, J, p, num_Hs_r, num_EV, save_data=True)

                exec_time = time.time() - start_time
                et.append(exec_time)
                tqdm.write('{} | Computed: L={:02d} | num_EV={} | periodic={: <5}.'.format(dt(), L, num_EV, str(p))) #, flush=True)
                tqdm.write('{} | Execution took {: 8.2f}s or {: 6.2f}min.'.format(dt(), exec_time, exec_time/60)) #, flush=True)
                tqdm.write(' ') #, flush=True)

        tqdm.write('='*40) #, flush=True)
        tqdm.write(' ') #, flush=True)

    tqdm.write('{} | Batch {:03d} of {:d} completed.'.format(dt(), i+1, batches)) #, flush=True)
    tqdm.write('='*60) #, flush=True)
    tqdm.write(' ') #, flush=True)

    if check_shutdown_signal():
        break